In [1]:
import numpy as np
import pandas as pd

import plotly.offline as off
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [23]:
# Specify data file and load both dataframes from it
datafile = '/data/RocklandSample/derivatives/mca/mca.h5'
df = pd.read_hdf(datafile, 'graphs')

In [24]:
sub = df['sub'].value_counts().to_dict()
df_ = df.loc[df['noise_type'] == df['noise_type']]
_df = df.loc[df['noise_type'] != df['noise_type']].loc[df["os"] == "ubuntu"]

In [25]:
met = 'fro'
distance = '{0} (percent difference)'.format(met)

for ss in sub:
    ref = _df.loc[df['sub'] == ss].iloc[0]
    idx = df_.loc[df['sub'] == ss].index
    df_.loc[idx, distance] = 100.0 * df_.loc[idx, '{0} (ref)'.format(met)] / ref['{0} (self)'.format(met)]

In [26]:
png_outpath = '/data/RocklandSample/derivatives/mca/pngs/{0}_summary.png'.format(met)

title = "Differences in Monte Carlo Arithmetic Injected Structural Connectomes"
fig = px.violin(df_, y=distance, x='sub', color='noise_type',
                violinmode='overlay', points='all', box=False,
                hover_data=['simulation_id'], title=title)

fig.update_layout(title=title, height=1200, width=1600, font=dict(size=24))
fig.show()
fig.write_image(png_outpath)

In [27]:
# The below values were picked for the Frobenius norm

# ss = sorted(sub.keys())[5]
ss = 'A00057725'
print(ss)

df_ss = df_.query('sub == "{0}" and noise_type == "rr"'.format(ss))
modes = df_ss[distance].value_counts().sort_index().to_dict()
mkeys = sorted(modes.keys())
print(modes)

keylocs = [1, 2, 4, 5, 0, 8, 14, 15, 17]

refvol = 4

graphs_to_plot = []
subplot_titles = []
for k in keylocs:
    loc = mkeys[k]
    graphs_to_plot += [df_ss[df_ss[distance] == loc].iloc[0]['graph']]
    if k:
        subplot_titles += ["%d / 100 (%.3f %%)" % (modes[loc], loc)]
    else:
        subplot_titles += ["reference"]

A00057725
{0.0: 1, 0.2505266347888638: 1, 0.798719757255969: 1, 5.926997540101985: 1, 5.945413416377008: 2, 6.099911824313505: 6, 6.105300735638229: 1, 6.335178101439875: 2, 33.17413289436415: 14, 33.18123340295096: 2, 33.48109566972303: 5, 33.55890763632209: 8, 33.58206258387237: 4, 33.724807754843795: 1, 34.34231718561546: 13, 34.35412687906012: 6, 34.61914747269582: 3, 55.63978195550777: 16, 55.64567060219096: 5, 55.83316496004805: 8}


In [28]:
png_outpath = '/data/RocklandSample/derivatives/mca/pngs/{0}_{1}'.format(met, ss)

axis_template = dict(showgrid = False, zeroline = False,
                     showticklabels = False, ticks = '')

ref_template = dict(mirror=True, showline=True,
                    linewidth=6, linecolor='gray')

fig = make_subplots(rows=3, cols=3,
                    vertical_spacing=0.05,
                    horizontal_spacing=0.03,
                    subplot_titles=subplot_titles)

for idx, gtp in enumerate(graphs_to_plot):
    r, c = int((idx)/ 3) + 1, int((idx) % 3)+1 
    modif = '' if not idx else str(idx + 1)
    if idx == refvol:
        dat = np.log10(gtp+1)
    else:
        dat = gtp - graphs_to_plot[refvol]
        dat = np.sign(dat)*np.log10(np.abs(dat) + 1)
#         dat = np.nan_to_num((gtp - graphs_to_plot[refvol]) / (graphs_to_plot[refvol] + 1))
#         dat = dat/graphs_to_plot[refvol]
#         dat = np.sign(dat)*np.log10(np.abs(dat))

    trace = go.Heatmap(z=dat, coloraxis="coloraxis")

#     single = go.Figure(data=trace)
#     single.update_layout(yaxis=axis_template, )
#     single.write_image(png_outpath + '_{0}.png'.format(idx))

    fig.append_trace(trace, row=r, col=c)
    for k in axis_template:
        fig['layout']['xaxis'+modif][k] = axis_template[k]
        fig['layout']['yaxis'+modif][k] = axis_template[k]
    
    if idx == refvol:
        for k in ref_template:
            fig['layout']['xaxis'+modif][k] = ref_template[k]
            fig['layout']['yaxis'+modif][k] = ref_template[k]

    fig['layout']['yaxis'+modif]['autorange'] = "reversed"

title = 'Error-Induced Deviations from Reference Connectome (Subject {0})'.format(ss)
fig.update_coloraxes(dict(colorscale=px.colors.diverging.RdBu[::-1],
                          showscale=True,
                          cmin=-3,
                          cmid=0,
                          cmax=3,
                          colorbar=dict(tickvals=[-3, 0, 3],
                                        title="Connectivity Strength",
                                        ticktext=['higher',
                                                  'no difference',
                                                  'lower'])))

fig.update_layout(title=title, height=1500, width=1600, font=dict(size=26))
for idx, anno in enumerate(fig.layout.annotations):
    fig.layout.annotations[idx]['font']['size'] = 28
fig.show()
fig.write_image(png_outpath + '_panel.png')